In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/prateek1736/WineQuality.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="prateek1736"
os.environ["MLFLOW_TRACKING_PASSWORD"]="04553a50d21f8b3844ce19ec04ab041d4aa1ed22"

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    

In [5]:
from ML_PROJECT.constants import *
from ML_PROJECT.utils.common import create_directories, read_yaml
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/prateek1736/WineQuality.mlflow",
           
        )

        return model_evaluation_config

In [6]:
from pathlib import Path
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlflow.models.signature import infer_signature
from ML_PROJECT.entity.config_entity import ModelEvaluationConfig
from ML_PROJECT.utils.common import save_json


In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Prepare MLflow tracking
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Predict
            predicted_qualities = model.predict(test_x)

            # Calculate metrics
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Prepare input example and signature
            input_example = test_x.iloc[:1]
            signature = infer_signature(test_x, predicted_qualities)

            # Log the model with input example and signature
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(
                    model,
                    "model",
                    registered_model_name="ElasticnetModel",
                    input_example=input_example,
                    signature=signature
                )
            else:
                mlflow.sklearn.log_model(
                    model,
                    "model",
                    input_example=input_example,
                    signature=signature
                )

In [8]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

Tracking URI: https://dagshub.com/prateek1736/WineQuality.mlflow
[2025-05-25 06:16:05,352: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-25 06:16:05,355: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-25 06:16:05,358: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-25 06:16:05,359: INFO: common: created directory at: artifacts]
[2025-05-25 06:16:05,360: INFO: common: created directory at: artifacts/model_evaluation]
[2025-05-25 06:16:06,162: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/05/25 06:16:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 4
Created version '4' of model 'ElasticnetModel'.


🏃 View run delicate-bat-434 at: https://dagshub.com/prateek1736/WineQuality.mlflow/#/experiments/0/runs/b2c5253e4f80481d88175970b3b6ae89
🧪 View experiment at: https://dagshub.com/prateek1736/WineQuality.mlflow/#/experiments/0
